In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import os
import warnings

In [9]:
# Reading the CEMS CSV file
AR_CEMS = pd.read_csv("TESTING.csv")

# Reading the CEMS CSV file
epa_crosswalk = pd.read_csv("EPA_crosswalk.csv")

# Convert the 'date' column to datetime data type
AR_CEMS['date'] = pd.to_datetime(AR_CEMS['date'])

In [10]:
# Converting 'EIA_PLANT_ID' to int64
epa_crosswalk['EIA_PLANT_ID'] = pd.to_numeric(epa_crosswalk['EIA_PLANT_ID'], errors='coerce').astype('Int64')
epa_crosswalk.dropna(subset=['EIA_PLANT_ID'], inplace=True)
epa_crosswalk['EIA_PLANT_ID'] = pd.to_numeric(epa_crosswalk['EIA_PLANT_ID'], errors='coerce').astype('int64')

# Converting 'CAMD_UNIT_ID' and 'EIA_GENERATOR_ID' to <class 'str'>
epa_crosswalk['CAMD_UNIT_ID'] = epa_crosswalk['CAMD_UNIT_ID'].astype(str)
epa_crosswalk['EIA_GENERATOR_ID'] = epa_crosswalk['EIA_GENERATOR_ID'].astype(str)
AR_CEMS['unitId'] = AR_CEMS['unitId'].astype(str)


# Merging EPA Crosswalk and CEMS
AR_CEMS = pd.merge( AR_CEMS, 
                    epa_crosswalk, 
                    left_on=['facilityId', 'unitId'], 
                    right_on=['CAMD_PLANT_ID', 'CAMD_UNIT_ID'], 
                    how='left')


# Dropping duplicate columns
AR_CEMS.drop(columns=['facilityId', 'unitId','CAMD_PLANT_ID', 'CAMD_UNIT_ID'], inplace=True)


# Bringing 'EIA_PLANT_ID' and 'EIA_GENERATOR_ID' to the front
AR_CEMS.insert(2, 'EIA_PLANT_ID', AR_CEMS.pop('EIA_PLANT_ID'))
AR_CEMS.insert(3, 'EIA_GENERATOR_ID', AR_CEMS.pop('EIA_GENERATOR_ID'))

AR_CEMS.head()

,stateCode,facilityName,EIA_PLANT_ID,EIA_GENERATOR_ID,associatedStacks,date,hour,opTime,grossLoad,steamLoad,...,EIA_LATITUDE,EIA_LONGITUDE,EIA_RETIRE_YEAR,PLANT_ID_CHANGE_FLAG,MOD_EIA_PLANT_ID,MOD_EIA_BOILER_ID,MOD_EIA_GENERATOR_ID_BOILER,MOD_EIA_GENERATOR_ID_GEN,MATCH_TYPE_GEN,MATCH_TYPE_BOILER
0,AR,Thomas Fitzhugh,201,1,NaN,2010-01-01,0,1.0,98.0,NaN,...,35.462382,-93.804926,0.0,0.0,201.0,2,1,1,3_1_Generator (generators) match on plant and ...,6_1_EnviroAssoc (boilers and generators) match...
1,AR,Thomas Fitzhugh,201,2,NaN,2010-01-01,0,1.0,98.0,NaN,...,35.462382,-93.804926,0.0,0.0,201.0,NaN,NaN,2,3_1_Generator (generators) match on plant and ...,NaN
2,AR,Thomas Fitzhugh,201,1,NaN,2010-01-01,1,1.0,97.0,NaN,...,35.462382,-93.804926,0.0,0.0,201.0,2,1,1,3_1_Generator (generators) match on plant and ...,6_1_EnviroAssoc (boilers and generators) match...
3,AR,Thomas Fitzhugh,201,2,NaN,2010-01-01,1,1.0,97.0,NaN,...,35.462382,-93.804926,0.0,0.0,201.0,NaN,NaN,2,3_1_Generator (generators) match on plant and ...,NaN
4,AR,Thomas Fitzhugh,201,1,NaN,2010-01-01,2,1.0,98.0,NaN,...,35.462382,-93.804926,0.0,0.0,201.0,2,1,1,3_1_Generator (generators) match on plant and ...,6_1_EnviroAssoc (boilers and generators) match...


In [11]:
# Ignore all warnings
warnings.filterwarnings("ignore")

# Create an empty list to store the DataFrames
generators = []


generators_2010 = pd.read_csv("./2010.csv")
generators_2011 = pd.read_csv("./2011.csv")
generators_2012 = pd.read_csv("./2012.csv")


# ------------- Renaming columns and renaming titles for generators_2010 ------------------
generators_2010.columns.values[0] = "Utility ID"
generators_2010.columns.values[2] = "Plant Code"
generators_2010.columns.values[6] = "Generator ID"

columns_to_keep = [
        'Utility ID', 'UTILITY_NAME', 'Plant Code', 'PLANT_NAME', 'STATE', 'COUNTY',
        'Generator ID', 'NAMEPLATE', 'SUMMER_CAPABILITY', 'WINTER_CAPABILITY', 'ENERGY_SOURCE_1'
    ]
    generators_2010 = generators_2010.filter(items=columns_to_keep)


    generators_2010.rename(columns={
        'UTILITY_NAME': 'Utility Name',
        'PLANT_NAME': 'Plant Name',
        'STATE': 'State',
        'COUNTY': 'County',
        'NAMEPLATE': 'Nameplate',
        'WINTER_CAPABILITY': 'Winter Capability',
        'SUMMER_CAPABILITY': 'Summer Capability',
        'ENERGY_SOURCE_1': 'Energy Source 1'
    }, inplace=True)
# -----------------------------------------------------------------------------------------



# ------------- Renaming columns and renaming titles for generators_2011 ------------------
# Update the column names
generators_2011.rename(columns=lambda x: x if not x.startswith("Unnamed:") else generators_2011.iloc[0][x], inplace=True)

# Drop the first row since it is now the column names
generators_2011 = generators_2011[1:].reset_index(drop=True)

generators_2011.columns.values[0] = "Utility ID"
generators_2011.columns.values[2] = "Plant Code"
generators_2011.columns.values[6] = "Generator ID"

generators.append(generators_2011)
# -----------------------------------------------------------------------------------------



# ------------- Renaming columns and renaming titles for generators_2012 ------------------
# Update the column names
generators_2012.rename(columns=lambda x: x if not x.startswith("Unnamed:") else generators_2012.iloc[0][x], inplace=True)

# Drop the first row since it is now the column names
generators_2012 = generators_2012[1:].reset_index(drop=True)

generators_2012.columns.values[0] = "Utility ID"
generators_2012.columns.values[2] = "Plant Code"
generators_2012.columns.values[6] = "Generator ID"
generators.append(generators_2012)
# -----------------------------------------------------------------------------------------


# Folder path containing the CSV files
folder_path = "Generators/"


# --------- TO DEBUG ---------------------------------------------------------------
i = 0
for filename in os.listdir(folder_path):
    if (folder_path + filename) != 'Generators/.DS_Store': 
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(folder_path + filename)
        
        print(filename)
        
        # Update the column names
        df.rename(columns=lambda x: x if not x.startswith("Unnamed:") else df.iloc[0][x], inplace=True)

        # Drop the first row since it is now the column names
        df = df[1:].reset_index(drop=True)
        
        # Convert to int64
        df['Plant Code'] = df['Plant Code'].apply(lambda x: pd.to_numeric(x, errors='coerce', downcast='integer'))
        df.dropna(subset=['Plant Code'], inplace=True)
        df['Plant Code'] = df['Plant Code'].astype('int64')
        
        # Manually renaming first column
        df.columns.values[0] = "Utility ID"

        # Append the DataFrame to the list
        generators.append(df)
        
# ---------------------------------------------------------------------------------

        
# For some reason, 2019 and 2020 has Plant Code as "Plant_Code", so it was manually changed 
generators[6].columns.values[2] = "Plant Code"
generators[7].columns.values[2] = "Plant Code"

generators[6].columns.values[6] = "Generator ID"
generators[7].columns.values[6] = "Generator ID"

2020.csv
2019.csv
2018.csv
2015.csv
2014.csv
2016.csv
2017.csv
2013.csv
2012.csv
2010.csv


KeyError: 'Plant Code'

In [6]:
# ------------- Removing unecessary columns & renaming column titles for 2010 and 2011 ------------------

for i in range(2):
    columns_to_keep = [
        'Utility ID', 'UTILITY_NAME', 'Plant Code', 'PLANT_NAME', 'STATE', 'COUNTY',
        'Generator ID', 'NAMEPLATE', 'SUMMER_CAPABILITY', 'WINTER_CAPABILITY', 'ENERGY_SOURCE_1'
    ]
    generators[i] = generators[i].filter(items=columns_to_keep)


    generators[i].rename(columns={
        'UTILITY_NAME': 'Utility Name',
        'PLANT_NAME': 'Plant Name',
        'STATE': 'State',
        'COUNTY': 'County',
        'NAMEPLATE': 'Nameplate',
        'WINTER_CAPABILITY': 'Winter Capability',
        'SUMMER_CAPABILITY': 'Summer Capability',
        'ENERGY_SOURCE_1': 'Energy Source 1'
    }, inplace=True)
# --------------------------------------------------------------------------------------------------------


'''
# ------------- Removing unecessary columns & renaming column titles -------------------------------------

columns_to_keep = [
    'Utility ID', 'Utility Name', 'Plant Code', 'Plant Name', 'State', 'County',
    'Generator ID', 'Nameplate Capacity (MW)', 'Summer Capacity (MW)', 'Winter Capacity (MW)', 'Energy Source 1'
]
generators[10] = generators[10].filter(items=columns_to_keep)

generators[10].rename(columns={
    'Nameplate Capacity (MW)': 'Nameplate',
    'Summer Capacity (MW)': 'Winter Capability',
    'Winter Capacity (MW)': 'Summer Capability',
}, inplace=True)

# --------------------------------------------------------------------------------------------------------


generators[10].to_csv('2020_new.csv', index=False)
# generators[6].to_csv('2016_new.csv', index=False)
# generators[7].to_csv('2017_new.csv', index=False)
'''

,Utility ID,Plant Code,Generator ID
0,7,10867,GEN1
1,8,50903,GEN1
2,21,10671,GEN1
3,21,10671,GEN2
4,25,2527,4


In [ ]:
merged_tables_list = []

for i in range(2013, 2020):
    # Filter the 'date' column for the specific year (i)
    AR_filtered = AR_CEMS[AR_CEMS['date'].dt.year == i]

    # Merge the filtered DataFrame with the corresponding generator DataFrame
    merged_table = pd.merge(AR_filtered, 
                            generators[i - 2013], 
                            left_on=['EIA_PLANT_ID', 'EIA_GENERATOR_ID'], 
                            right_on=['Plant Code', 'Generator ID'], how='left')
    
    # Append the merged DataFrame to the list
    merged_tables_list.append(merged_table)
    
combined_df = pd.concat(merged_tables_list, ignore_index=True)

# Count the number of rows with missing values in 'Plant Code' and 'Generator ID' columns
rows_not_matched = combined_df['Plant Code'].isnull().sum() + combined_df['Generator ID'].isnull().sum()

print("Number of rows not matched:", rows_not_matched)

In [9]:
generators_2010 = pd.read_csv("Generators_Cleaned/2012.csv")

generators_2010.drop(columns=['Utility ID'], inplace=True)


generators_2010.to_csv('2012_new.csv', index=False)

